In [16]:
#imports
import os
import gradio as gr
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings,ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from docs import chunks

In [17]:
# variables
DB_NAME = 'ollama-vectordb'
MODEL = 'llama3.2:latest'
EMBED_MODEL = 'nomic-embed-text'

## Create Chroma vector DB

In [18]:
#initialise embeddings
embeddings = OllamaEmbeddings(model = EMBED_MODEL) 

In [19]:
# create new Chroma datastore
if os.path.exists(DB_NAME):
    Chroma(persist_directory= DB_NAME, embedding_function= embeddings).delete_collection()
    
# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=DB_NAME)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")
    

Vectorstore created with 135 documents


In [20]:
#quick test on similarity search using the embeddings
docs = vectorstore.similarity_search('What is the Reinsurance product in insurellm',k = 2)
docs

[Document(id='e6d748d5-144d-4f06-832a-57a64d236a59', metadata={'doc_type': 'company', 'source': 'knowledge-base\\company\\overview.md'}, page_content='# Overview of Insurellm\n\nInsurellm is an innovative insurance tech firm with 200 employees across the US.\nInsurellm offers 4 insurance software products:\n- Carllm, a portal for auto insurance companies\n- Homellm, a portal for home insurance companies\n- Rellm, an enterprise platform for the reinsurance sector\n- Marketllm, a marketplace for connecting consumers with insurance providers\n  \nInsurellm has more than 300 clients worldwide.'),
 Document(id='5743d5c5-07ce-4fe6-a185-564dd672af2e', metadata={'doc_type': 'contracts', 'source': 'knowledge-base\\contracts\\Contract with Apex Reinsurance for Rellm.md'}, page_content='# Contract with Apex Reinsurance for Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Terms\n\n1. **Parties Involved**: This contract (“Agreement”) is entered into between Insurellm, Inc. (“Provider”) and A

In [21]:
vectorstore.similarity_search('Who is the data scientist',k = 2)

[Document(id='c23eaa54-a9d2-40af-b5d2-15261a06f974', metadata={'doc_type': 'employees', 'source': 'knowledge-base\\employees\\Samuel Trenton.md'}, page_content='# HR Record\n\n# Samuel Trenton\n\n## Summary\n- **Date of Birth:** April 12, 1989  \n- **Job Title:** Senior Data Scientist  \n- **Location:** Austin, Texas  \n\n## Insurellm Career Progression\n- **January 2020 - Present:** Senior Data Scientist  \n  *Promoted for demonstrating exceptional analytical skills and leadership potential. Led several projects that improved customer segmentation strategies, resulting in a 15% increase in customer retention.*\n\n- **June 2018 - December 2019:** Data Scientist  \n  *Joined the Insurellm team and worked on developing predictive modeling techniques to assess risk for both B2B and B2C customers. Received recognition for the success of the "Risk Assessment Model" project.*\n\n- **August 2016 - May 2018:** Junior Data Analyst  \n  *Started at Insurellm as a Junior Data Analyst, focusing on

In [22]:
# fetch sample vector and view its dimensions
collection = vectorstore._collection
sample_embedding = collection.get(limit = 1, include = ['embeddings'])['embeddings'][0]
dimensions = len(sample_embedding)
print("Nos of dims: ",dimensions)

Nos of dims:  768


In [23]:
print(sample_embedding)

[-1.03440592e-02  5.65509312e-02 -2.19752684e-01  2.61731502e-02
  4.21056487e-02 -7.40505150e-03 -1.54459160e-02 -3.32542975e-03
  3.75442095e-02 -4.17945459e-02 -2.86805723e-02  7.85878301e-03
  8.28650296e-02 -2.24776138e-02  9.18852817e-03  1.31700635e-02
 -2.20770040e-03 -4.49386705e-03 -2.04564016e-02 -1.80187039e-02
 -2.85939071e-02 -5.36701232e-02 -5.65863512e-02 -3.58319506e-02
  6.27389774e-02  3.38080265e-02  1.13324844e-03  1.22766579e-02
  5.79013564e-02  4.45616283e-02  7.64356628e-02 -2.58082636e-02
 -3.20745856e-02 -5.91641255e-02 -2.87206657e-02 -7.83755071e-03
  2.72876713e-02  3.27297710e-02 -1.90202072e-02  1.37488255e-02
  6.53089955e-02  2.01688260e-02  9.02511645e-03 -5.77179566e-02
  5.97879440e-02  3.90188694e-02 -2.04071328e-02  2.81482656e-02
  3.96651626e-02 -5.12509122e-02  6.11025980e-03 -6.63496880e-03
  2.04831883e-02 -4.31475171e-04  4.47583608e-02 -4.17908952e-02
 -5.51801808e-02  3.74281593e-02 -2.52172984e-02 -1.81249715e-02
  7.10056648e-02  3.53955

## Langchain components

In [24]:
# Initialize LangChain components

# initialise response llm
llm = ChatOllama(model=MODEL, temperature=0.7)

# initialise vectorstore retriever
embeddings = OllamaEmbeddings(model= EMBED_MODEL)
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_type = 'mmr',search_kwargs = {'k':4})

# setting up conversation memory for chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [25]:
# putting all the pieces for conversational chain
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [26]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

## Gradio Chat Interface

In [27]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

## Debugging the LLM
**Question: Who is the Data Scientist of Insurellm**

In [28]:
from langchain_core.callbacks import StdOutCallbackHandler

# initialise response llm
llm = ChatOllama(model=MODEL, temperature=0.7)

# initialise vectorstore retriever
embeddings = OllamaEmbeddings(model= EMBED_MODEL)
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_type = 'similarity',search_kwargs = {'k':5})

# setting up conversation memory for chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting all the pieces for conversational chain
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory,callbacks=[StdOutCallbackHandler()])

query = "Give me the pricing of all products of Insurellm"
result = conversation_chain.invoke({'question': query})
answer = result['answer']
print(answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
# Overview of Insurellm

Insurellm is an innovative insurance tech firm with 200 employees across the US.
Insurellm offers 4 insurance software products:
- Carllm, a portal for auto insurance companies
- Homellm, a portal for home insurance companies
- Rellm, an enterprise platform for the reinsurance sector
- Marketllm, a marketplace for connecting consumers with insurance providers
  
Insurellm has more than 300 clients worldwide.

### 6. Customer Portal
A user-friendly online portal and mobile application enables customers to manage their policies, submit claims, and view coverage details 24/7. Homellm prioritizes transparency and ease of use,